# Original Data

First we pull in the original data that we will transform to data we want to use. This is freely available from https://evolving-hockey.com/.

This is the game stats data: annual stats like goals, assists, games played, etc. 

Take a quick look at it before doing some matching with the contract player data. 

Also note that there are actually a few different tables that we are going to have to deal with here:

* Standard (even-strength, power play, ...)

* On-Ice

* Relative_TM

* GAR

# Match Game Stats Data to Player Name Data

We want to match up as many players as possible between our two data sets.

Because they come from different sites, there are some naming discrepancies. Eg. Chris vs. Christopher. 

So without going through by hand, we are bound to lose a few in the matching process.

So we started with the last ~10 years of players so that we end up with a reasonably large sample still after we have matched as many players as possible. 

We'll also tack on the unique player ID to the game stats players at this point so we don't have to keep trying to match by name moving forward. 

In [1]:
from utils.game_tables import FormatGames
import pandas as pd

# scale data 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
tables = []
folder = './data/'
root = 'EH_Tables/Evolving_Hockey_'
date = '2019-06-27'
info_1_list = ['GAA','On-Ice','Relative_TM','Standard','Standard']
info_2_list = ['GAR','EV','EV','EV','PP']    
years = list(range(2007,2019))

In [3]:
fg = FormatGames('./data/player_stats.csv')
for ii in range(len(info_1_list)):
    
    # Get the right name for the file
    info_1 = info_1_list[ii]
    info_2 = info_2_list[ii]
    
    filename = folder+root
    
    if info_1 != 'GAA':
        filename += 'skater_stats_'
        
    filename+=info_1+'_'+info_2+'_'
    
    if info_1 == 'GAA':
        filename+='Skaters_'
        
    elif info_1=='Standard':
        filename+='no_adj_'
        
    else:
        filename+='adj_'
        
    filename+=date+'.csv'
    
    # Make a save file name
    savename = folder+'game_stats_'+info_1+'_'+info_2+'.csv'
    
    # Import data into df and modify to include the player_id
    print('----------------------')
    print('For {:s} - {:s} Table:'.format(info_1,info_2))
    fg.load_games_csv(filename, years)
    fg.format_games()
    fg.save_games_csv(savename)
    

----------------------
For GAA - GAR Table:

Number of unique players in PLAYER DF: 2516
Original number of unique players in GAMES DF:  2042
Number of unique players in GAMES DF that match PLAYER DF: 1601
Total number of unmatched last names:  710
Total number of unmatched first names: 203

----------------------
For On-Ice - EV Table:

Number of unique players in PLAYER DF: 2516
Original number of unique players in GAMES DF:  2058
Number of unique players in GAMES DF that match PLAYER DF: 1614
Total number of unmatched last names:  699
Total number of unmatched first names: 201

----------------------
For Relative_TM - EV Table:

Number of unique players in PLAYER DF: 2516
Original number of unique players in GAMES DF:  2058
Number of unique players in GAMES DF that match PLAYER DF: 1614
Total number of unmatched last names:  699
Total number of unmatched first names: 201

----------------------
For Standard - EV Table:

Number of unique players in PLAYER DF: 2516
Original number of 

---

Okay so we lost 900 out of the ~2000 we started with. Could be better, could be worse. 

But it looks like a lot of these names are younger players (ie no contracts), or played few games or just irrelevant.

We certainly missed a few mis-spells, but we still have over 1000 players, and many players have multiple contracts so for the purposes of this study it will suffice. 

# Create Game Stats Table With Relevant Stats

Now, instead of having 5 different tables with an assortment of repeated stats, we want one big table with everyones stats (and only the relevants ones). 

In [16]:
game_stats_standard_EV = pd.read_csv('./data/game_stats_Standard_EV.csv')
game_stats_standard_PP = pd.read_csv('./data/game_stats_Standard_PP.csv')
game_stats_GAA_GAR = pd.read_csv('./data/game_stats_GAA_GAR.csv')
game_stats_onice_EV= pd.read_csv('./data/game_stats_On-Ice_EV.csv')
game_stats_Relative_TM_EV = pd.read_csv('./data/game_stats_Relative_TM_EV.csv')

In [17]:
# Remove all rows that have player_id = -1
game_stats_standard_EV = game_stats_standard_EV[game_stats_standard_EV.player_id!=-1].reset_index()
game_stats_standard_PP = game_stats_standard_PP[game_stats_standard_PP.player_id!=-1].reset_index()
game_stats_GAA_GAR = game_stats_GAA_GAR[game_stats_GAA_GAR.player_id!=-1].reset_index()
game_stats_onice_EV = game_stats_onice_EV[game_stats_onice_EV.player_id!=-1].reset_index()
game_stats_Relative_TM_EV = game_stats_Relative_TM_EV[game_stats_Relative_TM_EV.player_id!=-1].reset_index()

In [18]:
# First need an id column - need player ID AND year played 
game_stats_standard_EV['player_season']=game_stats_standard_EV.apply(lambda row: str(row.player_id) + '_'+str(row.season),axis=1)
game_stats_standard_PP['player_season']=game_stats_standard_PP.apply(lambda row: str(row.player_id) + '_'+str(row.season),axis=1)
game_stats_GAA_GAR['player_season']=game_stats_GAA_GAR.apply(lambda row: str(row.player_id) + '_'+str(row.season),axis=1)
game_stats_onice_EV['player_season']=game_stats_onice_EV.apply(lambda row: str(row.player_id) + '_'+str(row.season),axis=1)
game_stats_Relative_TM_EV['player_season']=game_stats_Relative_TM_EV.apply(lambda row: str(row.player_id) + '_'+str(row.season),axis=1)

In [19]:
# Might want to double check that the DF has unique ids at this point (already found a few that needed fixing)
def check_unique(df):
    unique_list = []
    for ii in range(len(df)):
        ps =df.loc[ii,'player_season']
        if ps not in unique_list:
            unique_list.append(ps)
        else:
            print(df.loc[ii,'player'],ps)

In [20]:
check_unique(game_stats_standard_PP)

In [21]:
# For standard strength stuff, we want their per 60 stats for some of them 
# Note that 'iBLK' isn't in the PP stats
per_60_list = ['G',
              'A1',
              'A2',
              'Points',
              'iSF',
              'iCF',
              'iBLK',
              'iHF',
              'iHA',
              'GIVE',
              'TAKE',
              'iPENT2',
              'iPEND2',
              'iPENT5',
              'iPEND5']

for stat in per_60_list:
    game_stats_standard_EV[stat+'_p60']=60*game_stats_standard_EV[stat]/game_stats_standard_EV['TOI']
    if stat!='iBLK':
        game_stats_standard_PP[stat+'_p60']=60*game_stats_standard_PP[stat]/game_stats_standard_PP['TOI']

In [22]:
# For the GAR table, also get avg TOI
game_stats_GAA_GAR['TOI_avg']=game_stats_GAA_GAR['TOI_all']/game_stats_GAA_GAR['GP']

In [23]:
# Also get FO win % 
game_stats_standard_EV['FOW_perc'] = 100*game_stats_standard_EV['FOW']/(game_stats_standard_EV['FOW']+game_stats_standard_EV['FOL'])
game_stats_standard_PP['FOW_perc'] = 100*game_stats_standard_PP['FOW']/(game_stats_standard_PP['FOW']+game_stats_standard_PP['FOL'])

In [24]:
# Need to rename the standard stats to EV/PP
columns = list(game_stats_standard_EV.columns)
columns_EV = [column+'_EV' if (column!='player_season') and (column!='player_id') and (column!='position') else column for column in columns]
game_stats_standard_EV.columns = columns_EV

columns = list(game_stats_standard_PP.columns)
columns_PP = [column+'_PP' if (column!='player_season') and (column!='player_id') else column for column in columns]
game_stats_standard_PP.columns = columns_PP

In [25]:
# Needed in every table
extra = ['player_season']

# The extra stats we care about from the 2 'standard' lists
standard_list = ['Sh_perc',
                 'FOW',
                 'FOL',
                 'FOW_perc',
                 'FO_diff']

# On ice stats
on_ice_list = ['onGF',
              'onGA',
              'onG_diff',
              'onxGF',
              'onxGA',
              'onxG_diff',
              'onSF',
              'onSA',
              'onS_diff',
              'onCF',
              'onCA',
              'onC_diff',
              'onSh_perc']

# Rel stats
rel_tm_list = ['rel_TM_GF_impact',
              'rel_TM_GA_impact',
              'rel_TM_G_diff_impact',
              'rel_TM_xGF_impact',
              'rel_TM_xGA_impact',
              'rel_TM_xG_diff_impact',
              'rel_TM_SF_impact',
              'rel_TM_SA_impact',
              'rel_TM_S_diff_impact',
              'rel_TM_CF_impact',
              'rel_TM_CA_impact',
              'rel_TM_C_diff_impact']

# GAR stats
gar_list = ['GP',
           'TOI_all',
           'TOI_avg',
           'TOI_EV',
           'TOI_PP',
           'TOI_SH',
           'EV_GAR',
           'PP_GAR',
           'SH_GAR',
           'Pens_GAR',
           'GAR',
           'WAR']

First let's match up the two standard tables.

There are quite a few players who show up in EV but not PP - so we want them to get 0's for all their PP stats. 

In [26]:
game_stats_standard_EV_list = [x+'_EV' for x in per_60_list] +\
                        [x+'_p60_EV' for x in per_60_list] +\
                        [x+'_EV' for x in standard_list]
game_stats_standard_PP_list = [x+'_PP' for x in per_60_list  if x!='iBLK'] +\
                        [x+'_p60_PP' for x in per_60_list if x!='iBLK'] +\
                        [x+'_PP' for x in standard_list]

In [27]:
game_stats_standard_EV = game_stats_standard_EV[['player_id','player_EV','season_EV','position']+extra+game_stats_standard_EV_list]
game_stats_standard_PP = game_stats_standard_PP[game_stats_standard_PP_list+extra]
game_stats_onice_EV = game_stats_onice_EV[on_ice_list+extra]
game_stats_GAA_GAR = game_stats_GAA_GAR[gar_list+extra]
game_stats_Relative_TM_EV = game_stats_Relative_TM_EV[rel_tm_list+extra]

In [28]:
# Join standard EV and standard PP
all_standard = game_stats_standard_EV.merge(game_stats_standard_PP, how='left', on='player_season')

# for players who didn't play enough PP time, make their stats 0
all_standard.fillna(0, inplace=True)

In [29]:
# Next, join up on ice and rel_tm
standard_onice = all_standard.merge(game_stats_onice_EV, how='left', on='player_season')
standard_on_ice_rel_tm = standard_onice.merge(game_stats_Relative_TM_EV, how='left', on='player_season')

In [30]:
# Finally, do the GAR table.
# This one is a slightly different size so just make sure it worked properly. Only take players that show in the other tables.
standard_on_ice_rel_tm_gaa = standard_on_ice_rel_tm.merge(game_stats_GAA_GAR, how='left',on='player_season')

In [31]:
# rename player and season to not include EV
standard_on_ice_rel_tm_gaa.rename(columns={'player_EV':'player','season_EV':'season'}, inplace=True)
standard_on_ice_rel_tm_gaa.head()

,player_id,player,season,position,player_season,G_EV,A1_EV,A2_EV,Points_EV,iSF_EV,...,TOI_avg,TOI_EV,TOI_PP,TOI_SH,EV_GAR,PP_GAR,SH_GAR,Pens_GAR,GAR,WAR
0,1649,AARON.JOHNSON,20072008,D,1649_20072008,0,0,1,1,13,...,13.860000,341.1,55.2,16.5,2.6,0.3,-0.1,-1.4,1.4,0.3
1,1440,AARON.ROME,20072008,D,1440_20072008,1,0,1,2,13,...,18.176471,259.8,20.0,28.0,-1.4,-0.6,0.3,0.3,-1.3,-0.2
2,1478,ADAM.BURISH,20072008,F,1478_20072008,3,3,1,7,62,...,11.665000,676.0,18.2,223.6,-3.6,-0.2,-1.9,0.9,-4.8,-0.9
3,1410,ADAM.FOOTE,20072008,D,1410_20072008,0,2,6,8,54,...,23.252703,1325.2,35.6,318.2,-1.8,-0.6,1.9,0.2,-0.5,-0.1
4,1105,ADAM.HALL,20072008,F,1105_20072008,2,1,2,5,35,...,11.871739,372.9,9.7,141.7,-4.4,0.2,-0.6,0.0,-4.7,-0.9


In [32]:
standard_on_ice_rel_tm_gaa.fillna(0,inplace=True)

In [33]:
# Save to file
standard_on_ice_rel_tm_gaa.to_csv('./data/game_stats.csv',index=False)

# Remove Players 

If they don't show up in all 3 of players, contracts, and game stats dfs. 

In [34]:
player_stats = pd.read_csv('./data/player_stats.csv')
contract_stats = pd.read_csv('./data/contract_stats.csv')
game_stats = pd.read_csv('./data/game_stats.csv')

contract_stats = contract_stats[contract_stats.contract_type == 'STANDARD CONTRACT']

In [35]:
player_ids = player_stats.player_id.values
contract_ids = contract_stats.player_id.values
game_ids = game_stats.player_id.values

new_player_stats = pd.DataFrame(columns=player_stats.columns)
new_contract_stats = pd.DataFrame(columns=contract_stats.columns)
new_game_stats = pd.DataFrame(columns=game_stats.columns)

in_all = True
for id_unique in player_ids:
    
    if id_unique not in contract_ids:
        in_all = False
    elif id_unique not in game_ids:
        in_all = False
        
    if in_all:
        new_player_stats = new_player_stats.append(player_stats[player_stats.player_id==id_unique])
        new_contract_stats = new_contract_stats.append(contract_stats[contract_stats.player_id==id_unique])
        new_game_stats = new_game_stats.append(game_stats[game_stats.player_id==id_unique])
        
    in_all = True

In [36]:
print('Player Stats New/Old')
print('Shape',player_stats.shape, new_player_stats.shape)
print('Unique IDs',player_stats.player_id.nunique(), new_player_stats.player_id.nunique())

print('\nContract Stats New/Old')
print('Shape', contract_stats.shape, new_contract_stats.shape)
print('Unique IDs', contract_stats.player_id.nunique(), new_contract_stats.player_id.nunique())

print('\nGame Stats New/Old')
print('Shape', game_stats.shape, new_game_stats.shape)
print('Unique IDs', game_stats.player_id.nunique(), new_game_stats.player_id.nunique())

Player Stats New/Old
Shape (2516, 10) (1376, 10)
Unique IDs 2516 1376

Contract Stats New/Old
Shape (4362, 8) (3997, 8)
Unique IDs 1594 1376

Game Stats New/Old
Shape (8412, 110) (7976, 110)
Unique IDs 1613 1376


In [37]:
new_player_stats.to_csv('./data/player_stats_matched.csv', index=False)
new_contract_stats.to_csv('./data/contract_stats_matched.csv', index=False)
new_game_stats.to_csv('./data/game_stats_matched.csv', index=False)

# Scale Data

The last thing we want to do is scale all of our data. 

* Many of the values are strictly positive. Use min-max scaling for these.
* percentages will just be done by dividing by 100 (so the max may not be 1).
* Some are both positive/negative. Use standard scaling here. 